Azure AI Agent - AI Search agent

Create Azure AI Search agent to search over the documents indexed using Azure AI search service

In [13]:
#install dependencies
#%pip install -r requirements.txt

In [16]:
import os
from datetime import datetime as pydatetime
from typing import Any, List, Dict
from dotenv import load_dotenv
# Azure AI Projects
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import ConnectionType
from azure.ai.projects.models import AzureAISearchTool, AzureAISearchQueryType


In [17]:
#rename credentials.txt file to credentials.env file before running this, and provide required config values
load_dotenv("credentials.env")

True

Check version of ai projects library

In [18]:
import azure.ai.projects
print(azure.ai.projects.__version__)

1.0.0b10


In [19]:
# Create Client and Load Azure AI Foundry project client
credential = DefaultAzureCredential()
project_client = AIProjectClient.from_connection_string(
    credential=credential,
    scope=['https://management.azure.com','https://management.core.windows.net','https://management.core.windows.net/','https://management.core.windows.net','https://management.azure.com/','https://management.azure.com'],
    conn_str=os.environ["AIPROJECT_CONNECTION_STRING"],   
)

In [20]:
# List all agents to see what's already available
all_agents = project_client.agents.list_agents().data
print(f"Found {len(all_agents)} agents:")
for agent in all_agents:
	print(f"- {agent.name} (ID: {agent.id})")

Found 7 agents:
- FabricLH2-DataAgent2 (ID: asst_JYvI8L1xuap9zqE5NLVVyvcC)
- Agent193 (ID: asst_KDVZmGnuwyUPAIB9CIHCleRy)
- Agent537 (ID: asst_dGh6zkzAf48NKw1Xyr4JkFtY)
- AISearchAgent (ID: asst_b3Fe2GACrZrmgtJbGHOl6EiM)
- AISearch_Agent (ID: asst_Y90HCxqazgc0LC9MRkDFWURu)
- Agent766 (ID: asst_TLIGQPj8JcsoQpGwJh5wjFTi)
- FabricLH2-DataAgent (ID: asst_yMSAqwYv2pCZs8xuSaNiAPET)


In [21]:

aisearch_connection_name = os.getenv("AISEARCH_CONNECTION_NAME")

aisearch_connection = project_client.connections.get(
    connection_name=aisearch_connection_name,    
    include_credentials=True,  # Optional. Defaults to "False".
)
print(aisearch_connection)

{
 "name": "aiworkshop2aisearch",
 "id": "/subscriptions/74a008ce-fe34-4b7a-a4e7-012f6ca145c8/resourceGroups/aiworkshop2/providers/Microsoft.MachineLearningServices/workspaces/aiworkshop2-project1/connections/aiworkshop2aisearch",
 "authentication_type": "AAD",
 "connection_type": "ConnectionType.AZURE_AI_SEARCH",
 "endpoint_url": "https://aiworkshop2-aisearch.search.windows.net",
 "key": null
 "token_credential": "REDACTED"
}



Create new AI search agent

In [22]:

conn_id = aisearch_connection.id
indexName = os.getenv("AISEARCH_INDEX_NAME")
print(conn_id)

# Initialize agent AI search tool and add the search index connection id
ai_search = AzureAISearchTool(
    index_connection_id=conn_id, index_name=indexName, query_type=AzureAISearchQueryType.SEMANTIC, top_k=3, filter=""
)

# Create agent with AI search tool and process assistant run

agent = project_client.agents.create_agent(
    model=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    name="AISearchAgent",
    instructions="You are a helpful assistant",
    tools=ai_search.definitions,
    tool_resources=ai_search.resources
)

/subscriptions/74a008ce-fe34-4b7a-a4e7-012f6ca145c8/resourceGroups/aiworkshop2/providers/Microsoft.MachineLearningServices/workspaces/aiworkshop2-project1/connections/aiworkshop2aisearch


In [23]:
input_prompt = "does perks plus cover gym membership?"

In [24]:
# Create thread for communication
thread = project_client.agents.create_thread()
print(f"Created thread, ID: {thread.id}")

# Create message to thread
# Remember to update the message with your data
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content=input_prompt,
)
print(f"Created message, ID: {message.id}")

Created thread, ID: thread_tEcxaNfBCWZTMziSpoApbBYE
Created message, ID: msg_xYsw0hiqmFTiKsbwefz9tBJC


In [25]:
# Create and process agent run in thread with tools
run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

# Fetch and log all messages
messages = project_client.agents.list_messages(thread_id=thread.id)
print(f"Messages: {messages}")

Run finished with status: RunStatus.COMPLETED
Messages: {'object': 'list', 'data': [{'id': 'msg_BlRH4wcZHvGp8xibTB5xjdqy', 'object': 'thread.message', 'created_at': 1747347286, 'assistant_id': 'asst_46s7YBsdt5Fvsyn4W9ns5Khh', 'thread_id': 'thread_tEcxaNfBCWZTMziSpoApbBYE', 'run_id': 'run_fkimZd26nAVR8A19jQMzhmsj', 'role': 'assistant', 'content': [{'type': 'text', 'text': {'value': "Perks Plus typically offers various employee benefits, including wellness programs, but coverage for gym memberships can vary depending on the specific plan or company policies. It's best to check directly with your HR department or the Perks Plus program details to understand what is covered regarding gym memberships.", 'annotations': []}}], 'attachments': [], 'metadata': {}}, {'id': 'msg_xYsw0hiqmFTiKsbwefz9tBJC', 'object': 'thread.message', 'created_at': 1747347282, 'assistant_id': None, 'thread_id': 'thread_tEcxaNfBCWZTMziSpoApbBYE', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'

Response recvd: 

In [26]:
agent_response = messages.data[0].content[0].text.value# Get the content of the first message]
print(agent_response)

Perks Plus typically offers various employee benefits, including wellness programs, but coverage for gym memberships can vary depending on the specific plan or company policies. It's best to check directly with your HR department or the Perks Plus program details to understand what is covered regarding gym memberships.


Clean up

In [27]:

# Delete the assistant when done
project_client.agents.delete_agent(agent.id)
print("Deleted agent")
project_client.agents.delete_thread(thread.id)
print("Deleted thread")

Deleted agent
Deleted thread
